<a href="https://colab.research.google.com/github/OndwelaT/OndwelaT.github.io/blob/main/TweetPreprocessing_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# imports
import random
import re
import requests
import sklearn
import string

import nltk
nltk.download('stopwords')
nltk.download('punkt')

# for coloring out outputs
from termcolor import colored
# for comparing strings
!pip install contractions
import contractions

import pandas as pd
from bs4 import BeautifulSoup

! pip install spacymoji
import spacymoji

import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
! pip install emoji
import emoji

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 2.3 MB/s eta 0:00:00




| feature to remove | examples |
----------|-----------
| Special characters | ' } { [ ] # @ ! * < > ~ |
| Out of vocabulary words | icebucketchallenge, wowwwww |
| Emoji | 👾🙏🏴󠁧󠁢󠁳󠁣󠁴󠁿🤷🏽🥰 |
| URLs | https://colab.research.google.com |
| known social media features | @mentions, RT, #hashtags |
| nonstandard casing | YIKES, dO sOMe PrEpROceSSinG
lower case



In [ ]:

df = pd.read_excel('/content/data_to_sort (21).xlsx')
df.head()

,author_id,text,id,username,edit_history_tweet_ids
0,1813746727125663745,By-elections are occurring in Johannesburg's W...,1828775722254815737,XArthurDent,['1828775722254815737']
1,1780386776793731072,@collenmashawane @PhophiRamathuba I heard fire...,1829272628986823060,TreesAfric4255,['1829272628986823060']
2,3044850202,The DA in Limpopo is concerned by the departme...,1829211511182795243,dalimlegis,['1829211511182795243']
3,1085786884566118400,South Africa continues to grapple with water s...,1830246392687648877,Newzroom405,['1830246392687648877']
4,2937098086,@AmaruT20 @AfricaFactsZone South Africa depend...,1830164210547868092,blessing_komon,['1830164210547868092']


In [ ]:
# Print the existing columns in the DataFrame
print(df.columns)

# Drop only columns that exist in the DataFrame
existing_columns_to_drop = [col for col in ['author_id','id','edit_history_tweet_ids','username'] if col in df.columns]
if existing_columns_to_drop:  # Check if there are any valid columns to drop
    df = df.drop(columns=existing_columns_to_drop)

df.head()

Index(['author_id', 'text', 'id', 'username', 'edit_history_tweet_ids'], dtype='object')


,text
0,By-elections are occurring in Johannesburg's W...
1,@collenmashawane @PhophiRamathuba I heard fire...
2,The DA in Limpopo is concerned by the departme...
3,South Africa continues to grapple with water s...
4,@AmaruT20 @AfricaFactsZone South Africa depend...


In [ ]:
# Removal of URLs
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)
df['text'] = df['text'].apply(remove_urls)
df.head()

,text
0,By-elections are occurring in Johannesburg's W...
1,@collenmashawane @PhophiRamathuba I heard fire...
2,The DA in Limpopo is concerned by the departme...
3,South Africa continues to grapple with water s...
4,@AmaruT20 @AfricaFactsZone South Africa depend...


In [ ]:
# prompt: removal of \n with empyt space, also include it in clean_text

def remove_newline(text):
  return text.replace('\n', ' ')

df['text'] = df['text'].apply(remove_newline)
df['clean_text'] = df['text'].apply(remove_newline)
df.head()


,text,clean_text
0,By-elections are occurring in Johannesburg's W...,By-elections are occurring in Johannesburg's W...
1,@collenmashawane @PhophiRamathuba I heard fire...,@collenmashawane @PhophiRamathuba I heard fire...
2,The DA in Limpopo is concerned by the departme...,The DA in Limpopo is concerned by the departme...
3,South Africa continues to grapple with water s...,South Africa continues to grapple with water s...
4,@AmaruT20 @AfricaFactsZone South Africa depend...,@AmaruT20 @AfricaFactsZone South Africa depend...


In [ ]:
# Removal of HTML tags
def remove_html_tags(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)
df['clean_text'] = df['clean_text'].apply(remove_html_tags)
df.head()

,text,clean_text
0,By-elections are occurring in Johannesburg's W...,By-elections are occurring in Johannesburg's W...
1,@collenmashawane @PhophiRamathuba I heard fire...,@collenmashawane @PhophiRamathuba I heard fire...
2,The DA in Limpopo is concerned by the departme...,The DA in Limpopo is concerned by the departme...
3,South Africa continues to grapple with water s...,South Africa continues to grapple with water s...
4,@AmaruT20 @AfricaFactsZone South Africa depend...,@AmaruT20 @AfricaFactsZone South Africa depend...


In [ ]:


# Expand contractions
def expand_contractions(text):
    return contractions.fix(text)

df['clean_text'] = df['clean_text'].apply(expand_contractions)
df.head()

,text,clean_text
0,By-elections are occurring in Johannesburg's W...,By-elections are occurring in Johannesburg's W...
1,@collenmashawane @PhophiRamathuba I heard fire...,@collenmashawane @PhophiRamathuba I heard fire...
2,The DA in Limpopo is concerned by the departme...,The DA in Limpopo is concerned by the departme...
3,South Africa continues to grapple with water s...,South Africa continues to grapple with water s...
4,@AmaruT20 @AfricaFactsZone South Africa depend...,@AmaruT20 @AfricaFactsZone South Africa depend...


In [ ]:


# Removal of @mentions
def remove_mentions(text):
    mention_pattern = re.compile(r'@\w+')
    return mention_pattern.sub(r'', text)

df['clean_text'] = df['clean_text'].apply(remove_mentions)
df.head()


,text,clean_text
0,By-elections are occurring in Johannesburg's W...,By-elections are occurring in Johannesburg's W...
1,@collenmashawane @PhophiRamathuba I heard fire...,I heard fire engines use petrol now to put o...
2,The DA in Limpopo is concerned by the departme...,The DA in Limpopo is concerned by the departme...
3,South Africa continues to grapple with water s...,South Africa continues to grapple with water s...
4,@AmaruT20 @AfricaFactsZone South Africa depend...,South Africa depend on other African nations...


In [ ]:

# Text normalization
def normalize_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove extra whitespace
    text = re.sub(' +', ' ', text)
    return text

df['clean_text'] = df['clean_text'].apply(normalize_text)
df.head()


,text,clean_text
0,By-elections are occurring in Johannesburg's W...,byelections are occurring in johannesburgs war...
1,@collenmashawane @PhophiRamathuba I heard fire...,i heard fire engines use petrol now to put ou...
2,The DA in Limpopo is concerned by the departme...,the da in limpopo is concerned by the departme...
3,South Africa continues to grapple with water s...,south africa continues to grapple with water s...
4,@AmaruT20 @AfricaFactsZone South Africa depend...,south africa depend on other african nations ...


In [ ]:

# Install the necessary library
!pip install googletrans==4.0.0-rc1

from googletrans import Translator

def translate_to_english(text):
  """Translates text to English."""
  translator = Translator()
  try:
    translated = translator.translate(text, dest='en')
    return translated.text
  except Exception as e:
    print(f"Translation error: {e}")
    return text  # Return the original text if translation fails

df['clean_text'] = df['clean_text'].apply(translate_to_english)
df.head()

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.7 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=d28334372996fa701af9d0b0b1599567dfa622b104ddbc22cc221cda28902a6e
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:


KeyboardInterrupt: 

In [ ]:
# prompt: tokenize text

# Tokenization
from nltk.tokenize import word_tokenize

def tokenize_text(text):
    return word_tokenize(text)

df['tokenized_text'] = df['clean_text'].apply(tokenize_text)
df.head()


In [ ]:
# Spelling corrections
!pip install pyspellchecker
from spellchecker import SpellChecker  # Import SpellChecker, not Spell
spell = SpellChecker()  # Correct the variable name

def correct_spelling(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())  # Find misspelled words
    print(misspelled_words)
    for word in text.split():
        if word in misspelled_words:
            correction = spell.correction(word)
            corrected_text.append(correction if correction is not None else word)  # Handle None corrections
        else:
            corrected_text.append(word)  # Keep correctly spelled words
    return " ".join(corrected_text)  # Join the words back into a string

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 16.7 MB/s eta 0:00:00


In [ ]:


# Assuming 'clean_text' is the column you want to correct
df['clean_text'] = df['clean_text'].apply(correct_spelling)

# Display the first few rows of the dataframe
df.head()


{'byelections', 'by…', 'johannesburgs'}
{'limpopo'}
{'department’s', 'limpopo', 'da', 'malwandla'}
{'using…', 'mbhele', 'eben', 'kaloola'}
{'😂😂😂', 'water😂😂', '😂😂'}
{'icj', 'etc'}
set()
{'governmenterected'}
{'km', 'gauteng', '–', 'usingx000da', 'makhaleng', '📸afdb'}
{'‘seriously’', 'africa’s', 'anc'}
{'byelections', 'pretorias', 'johannesburgs', 'crime…'}
{'sustainable…', 'longterm'}
{'5km', 'mpumalanga'}
{'5km', 'mpumalanga', 'raf'}
{'lagos', 'cairo', 'johannesburg', 'tinubu'}
{'byelections', 'by…', 'johannesburgs'}
{'byelections', 'pretorias', 'johannesburgs', 'crime…'}
{'vaal', 'africas', 'sa'}
{'america…', 'london…', 'africa…', 'eh…'}
{'change—every'}
set()
{'brics', 'banknew', 'africas', 'un'}
{'watercrisis', 'womens', 'southafric', 'childrens', 'worldnews', 'tatarstan', '44b'}
{'uk', 'poluted', 'polititians', 'eu', 'asia'}
{'uk', 'poluted', 'polititians', 'eu', 'asia'}
{'uk', 'poluted', 'polititians', 'eu', 'asia'}
{'uk', 'poluted', 'polititians', 'eu', 'asia'}
{'uk', 'poluted', 

,text,clean_text
0,By-elections are occurring in Johannesburg's W...,elections are occurring in johannesburgs ward ...
1,@collenmashawane @PhophiRamathuba I heard fire...,i heard fire engines use petrol now to put out...
2,The DA in Limpopo is concerned by the departme...,the a in limpopo is concerned by the departmen...
3,South Africa continues to grapple with water s...,south africa continues to grapple with water s...
4,@AmaruT20 @AfricaFactsZone South Africa depend...,south africa depend on other african nations t...


In [ ]:
#  removal of emojis

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F700-\U0001F77F"  # alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                           u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-B
                           u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           u"\U0001FA00-\U0001FA6F"  # Symbols and Pictographs Extended-A
                           u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-B
                           u"\U00002702-\U000027B0"  # Dingbats
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

df['clean_text'] = df['clean_text'].apply(remove_emoji)
df.head()


,text,clean_text
0,By-elections are occurring in Johannesburg's W...,elections are occurring in johannesburgs ward ...
1,@collenmashawane @PhophiRamathuba I heard fire...,i heard fire engines use petrol now to put out...
2,The DA in Limpopo is concerned by the departme...,the a in limpopo is concerned by the departmen...
3,South Africa continues to grapple with water s...,south africa continues to grapple with water s...
4,@AmaruT20 @AfricaFactsZone South Africa depend...,south africa depend on other african nations t...


In [ ]:
#  removal of stopwords

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    word_tokens = word_tokenize(text)
    filtered_text = [w for w in word_tokens if not w in stop_words]
    return " ".join(filtered_text)

df['clean_text'] = df['clean_text'].apply(lambda text: remove_stopwords(text))
df.head(

,text,clean_text
0,The DA-led @CityofCT's Water &amp; Sanitation ...,dated water amp sanitation directorate investi...
1,@NiranetheGOAT @BIGBLAKMIKE @badandwealthy @MU...,running water every tap house need shit fro de...
2,Durban: Two beaches are currently closed for s...,urban two beaches currently closed swimming du...
3,Durban: Two beaches are currently closed for s...,urban two beaches currently closed swimming du...
4,#Africa #SouthAfrica #KZN #Durban @trave...,africa southafrica kin urban


In [ ]:
# Lemmatization and POS Tagging

import spacy

nlp = spacy.load("en_core_web_sm")

def lemmatize_and_pos_tag(text):
  doc = nlp(text)
  return [(token.lemma_, token.pos_) for token in doc]

df['lemmatized_pos_tags'] = df['clean_text'].apply(lemmatize_and_pos_tag)
df.head()


,text,clean_text,lemmatized_pos_tags
0,The DA-led @CityofCT's Water &amp; Sanitation ...,the dated s water amp sanitation directorate i...,"[(the, DET), (date, VERB), (s, PROPN), (water,..."
1,@NiranetheGOAT @BIGBLAKMIKE @badandwealthy @MU...,there is running water in every tap in my hous...,"[(there, PRON), (be, VERB), (run, VERB), (wate..."
2,Durban: Two beaches are currently closed for s...,urban two beaches are currently closed for swi...,"[(urban, ADJ), (two, NUM), (beach, NOUN), (be,..."
3,Durban: Two beaches are currently closed for s...,urban two beaches are currently closed for swi...,"[(urban, ADJ), (two, NUM), (beach, NOUN), (be,..."
4,#Africa #SouthAfrica #KZN #Durban @trave...,africa southafrica kin urban,"[(africa, PROPN), (southafrica, PROPN), (kin, ..."


In [ ]:

# Removal of numbers
def remove_numbers(text):
    text = re.sub(r'\d+', '', text)
    return text

df['clean_text'] = df['clean_text'].apply(remove_numbers)
df.head()


,text,clean_text
0,By-elections are occurring in Johannesburg's W...,elections are occurring in johannesburgs ward ...
1,@collenmashawane @PhophiRamathuba I heard fire...,i heard fire engines use petrol now to put out...
2,The DA in Limpopo is concerned by the departme...,the a in limpopo is concerned by the departmen...
3,South Africa continues to grapple with water s...,south africa continues to grapple with water s...
4,@AmaruT20 @AfricaFactsZone South Africa depend...,south africa depend on other african nations t...


In [ ]:
# Create a new DataFrame with only the 'clean_text' column
clean_text_df = df[['clean_text']]

# Save the new DataFrame to an Excel file
clean_text_df.to_excel('/content/clean_text_data.xlsx', index=False)
